<a href="https://colab.research.google.com/github/usrmaia/Mundo-de-Wumpus/blob/main/Mundo_de_Wumpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import random, randint, choice
!pip install tabulate # Exibe dados em uma grid
from tabulate import tabulate
!pip install colorama # Dar cor ao texto
from colorama import Fore, init, deinit
!pip install python-sat
from pysat.solvers import Glucose3
!pip install db-sqlite3
import sqlite3

# Base de Conhecimento

In [ ]:
class BaseDeConhecimento:
  def __init__(self):
    self.conn = sqlite3.connect("base_de_conhecimento")
    self.cursor = self.conn.cursor()
    self.executarComando(self, "drop table base_de_conhecimento")
    self.executarComando(self, """
      create table if not exists base_de_conhecimento (
        tempo integer primary key autoincrement,
        conhecimento varchar,
        raciocinio varchar,
        acao varchar
      )
    """)
  
  @staticmethod
  def executarComando(self, comando):
    try:
      self.cursor.execute(comando)
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
    
  def imprimirTabela(self):
    try:
      self.cursor.execute(f"select * from base_de_conhecimento")
      sentencas = self.cursor.fetchall()
      header = ["Tempo", "Conhecimento", "Raciocínio", "Ação"]
      print(tabulate(sentencas, tablefmt="simple", stralign="center", headers=header))
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

  def retornarTabela(self):
    try:
      self.cursor.execute(f"select * from conhecimento")
      tabela = self.cursor.fetchall()
      return tabela
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
      return False

  def inserirLinha(self):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (conhecimento, raciocinio, acao) values ('', '', '')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

  def inserirConhecimento(self, conhecimento):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (conhecimento) values ('{conhecimento}')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
  
  def inserirRaciocinio(self, raciocinio):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (raciocinio) values ('{raciocinio}')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
  
  def inserirAcao(self, acao):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (acao) values ('{acao}')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
  
  def retornarConhecimento(self):
    try:
      self.cursor.execute(f"select conhecimento from base_de_conhecimento")
      tabela = self.cursor.fetchall()
      conhecimento = []
      for atomo in tabela:
        if atomo[0]: conhecimento.append(atomo[0])
      return conhecimento
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
      return False
  
  def retornarRaciocinio(self):
    try:
      self.cursor.execute(f"select raciocinio from base_de_conhecimento")
      tabela = self.cursor.fetchall()
      raciocinio = []
      for atomo in tabela:
        if atomo[0]: raciocinio.append(atomo[0])
      return raciocinio
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
      return False

# Geração do Mapa
Para cada posição temos **((wumpus, poco, ouro, parede), (fedor, brisa, brilho, impacto, grito))** atribuído de forma aleatória.

Esse dado (tupla) será captado pelo sensor.
### Exemplos:
* Posição segura: ((0, 0, 0, 0), (0, 0, _, _, _))
* Posição com 3 poços adjacentes: ((0, 0, 0, 0), (_, 3, _, _, _))

## Declaração
mapa = Mapa(linhas, colunas, quantidade_wumpus, quantidade_pocos, quantidade_ouros)

In [ ]:
class Mapa():
  def __init__(self, linhas, colunas, quantidade_wumpus, quantidade_pocos, quantidade_ouros):
    self.linhas = linhas
    self.colunas = colunas
    self.quantidade_wumpus = quantidade_wumpus
    self.quantidade_pocos = quantidade_pocos
    self.quantidade_ouros = quantidade_ouros
    self.taxa_wumpus = self.quantidade_wumpus / (self.linhas * self.colunas - 1 - self.quantidade_pocos - self.quantidade_ouros)
    self.taxa_pocos = self.quantidade_pocos / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_ouros)
    self.taxa_ouros = self.quantidade_ouros / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_pocos)
    self.status_posicao = {} # linha_coluna_posicao:((wumpus, poco, ouro, parede), (fedor, brisa, brilho, impacto, grito))
    self.mapa = []

  @staticmethod
  def mapearStatusDePosicao(self):
    for linha in range(-1, self.linhas + 1):
      for coluna in range(-1, self.colunas + 1):
        self.status_posicao[f"{linha}_{coluna}_posicao"] = ((0, 0, 0, 0), (0, 0, 0, 0, 0))
  
  def gerarMapa(self):
    self.mapa = []
    self.mapearStatusDePosicao(self)
    for linha in range(-1, self.linhas + 1):
      for coluna in range(-1, self.colunas + 1):
        if self.insercaoValida(self, linha, coluna):
          self.adicionarWumpusEFedor(self, linha, coluna)
          self.adicionarPocoEBrisa(self, linha, coluna)
          self.adicionarOuroEBrilho(self, linha, coluna)
        self.adicionarParedeEImpacto(self, linha, coluna)

  def imprimirMapa(self):
    self.mapa = []
    init(convert=True, autoreset=True) # colorama
    for linha in range(-1, self.linhas + 1):
      elementos_da_linha = []
      for coluna in range(-1, self.colunas + 1):
        elementos_da_celula = ""
        if self.possuiParede(self, linha, coluna):
          elementos_da_celula += (Fore.MAGENTA + "parede")
        if self.possuiImpacto(self, linha, coluna):
          elementos_da_celula += (Fore.MAGENTA + str(self.possuiImpacto(self, linha, coluna)) + "impacto")
        if self.possuiWumpus(self, linha, coluna):
          elementos_da_celula += (Fore.GREEN + "wumpus")
        if self.possuiPoco(self, linha, coluna):
          elementos_da_celula += (Fore.RED + "poco")
        if self.possuiOuro(self, linha, coluna):
          elementos_da_celula += (Fore.YELLOW + "ouro")
        if self.possuiFedor(self, linha, coluna):
          elementos_da_celula += (Fore.GREEN + str(self.possuiFedor(self, linha, coluna)) + "fedor")
        if self.possuiBrisa(self, linha, coluna):
          elementos_da_celula += (Fore.RED + str(self.possuiBrisa(self, linha, coluna)) + "brisa")
        if self.possuiBrilho(self, linha, coluna):
          elementos_da_celula += (Fore.YELLOW + "brilho")
        if self.possuiOK(self, linha, coluna):
          elementos_da_celula += (Fore.WHITE + "ok")
      
        elementos_da_linha.append(elementos_da_celula)
      self.mapa.append(elementos_da_linha)
    deinit() # colorama
    print(Fore.WHITE) # gambiarra
    print(tabulate(self.mapa, tablefmt="plain", stralign="center"))
    print(Fore.WHITE) # gambiarra
  
  def retornarStatusDaPosica(self, linha, coluna):
    try:
      return self.status_posicao[f"{linha}_{coluna}_posicao"]
    except:
      print("Posição inválida!")
      return False

  def pegarOuro(self, linha, coluna):
    ouro = self.status_posicao[f"{linha}_{coluna}_posicao"][0][2]
    if ouro:
      status = self.status_posicao[f"{linha}_{coluna}_posicao"]
      status = ((status[0][0], status[0][1], status[0][2] - 1, status[0][3]), (status[1][0], status[1][1], status[1][2] - 1, status[1][3], status[1][4]))
      self.status_posicao[f"{linha}_{coluna}_posicao"] = status
      return True
    return False

  @staticmethod
  def insercaoValida(self, linha, coluna):
    return not ((linha == 0 and coluna == 0) or (linha == -1 or linha == self.linhas or coluna == -1 or coluna == self.colunas))

  @staticmethod
  def adicionarWumpusEFedor(self, linha, coluna):
    if self.quantidade_wumpus >= 1:
      if random() < self.taxa_wumpus:
        self.adicionarWumpus(self, linha, coluna)
        self.adicionarFedor(self, linha - 1, coluna)
        self.adicionarFedor(self, linha, coluna - 1)
        self.adicionarFedor(self, linha + 1, coluna)
        self.adicionarFedor(self, linha, coluna + 1)
        self.quantidade_wumpus -= 1
  
  @staticmethod
  def adicionarPocoEBrisa(self, linha, coluna):
    if self.quantidade_pocos >= 1 and not self.possuiWumpus(self, linha, coluna):
      if random() < self.taxa_pocos:
        self.adicionarPoco(self, linha, coluna)
        self.adicionarBrisa(self, linha - 1, coluna)
        self.adicionarBrisa(self, linha, coluna - 1)
        self.adicionarBrisa(self, linha + 1, coluna)
        self.adicionarBrisa(self, linha, coluna + 1)
        self.quantidade_pocos -= 1
  
  @staticmethod
  def adicionarOuroEBrilho(self, linha, coluna):
    if self.quantidade_ouros >= 1 and not (self.possuiWumpus(self, linha, coluna) or self.possuiPoco(self, linha, coluna)):
      if random() < self.taxa_ouros:
        self.adicionarOuro(self, linha, coluna)
        self.adicionarBrilho(self, linha, coluna)
        self.quantidade_ouros -= 1
  
  @staticmethod
  def adicionarParedeEImpacto(self, linha, coluna):
    if (linha == -1 or linha == self.linhas) or (coluna == -1 or coluna == self.colunas):
      self.adicionarParede(self, linha, coluna)
    else:
      if (linha == 0 or linha == self.linhas - 1):
        self.adicionarImpacto(self, linha, coluna)
      if (coluna == 0 or coluna == self.colunas - 1):
        self.adicionarImpacto(self, linha, coluna)

  @staticmethod
  def adicionarWumpus(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0] + 1, status[0][1], status[0][2], status[0][3]), (status[1]))
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarPoco(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0], status[0][1] + 1, status[0][2], status[0][3]), (status[1]))
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarOuro(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0], status[0][1], status[0][2] + 1, status[0][3]), (status[1]))
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarParede(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0], status[0][1], status[0][2], status[0][3] + 1), (status[1])) 
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status

  @staticmethod
  def adicionarFedor(self, linha, coluna):
    if self.insercaoValida(self, linha, coluna) or (linha == 0 and coluna == 0):
      status = self.status_posicao[f"{linha}_{coluna}_posicao"]
      status = ((status[0]), (status[1][0] + 1, status[1][1], status[1][2], status[1][3], status[1][4]))
      self.status_posicao[f"{linha}_{coluna}_posicao"] = status

  @staticmethod
  def adicionarBrisa(self, linha, coluna):
    if self.insercaoValida(self, linha, coluna) or (linha == 0 and coluna == 0):
      status = self.status_posicao[f"{linha}_{coluna}_posicao"]
      status = ((status[0]), (status[1][0], status[1][1] + 1, status[1][2], status[1][3], status[1][4]))
      self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarBrilho(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0]), (status[1][0], status[1][1], status[1][2] + 1, status[1][3], status[1][4])) 
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarImpacto(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0]), (status[1][0], status[1][1], status[1][2], status[1][3] + 1, status[1][4])) 
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def possuiWumpus(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][0]:
      return True
    return False
  
  @staticmethod
  def possuiPoco(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][1]:
      return True
    return False
  
  @staticmethod
  def possuiOuro(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][2]:
      return True
    return False
  
  @staticmethod
  def possuiParede(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][3]:
      return True
    return False
  
  @staticmethod
  def possuiFedor(self, linha, coluna):
    fedor = self.status_posicao[f"{linha}_{coluna}_posicao"][1][0]
    if fedor:
      return fedor
    return False
  
  @staticmethod
  def possuiBrisa(self, linha, coluna):
    brisa = self.status_posicao[f"{linha}_{coluna}_posicao"][1][1]
    if brisa:
      return brisa
    return False
  
  @staticmethod
  def possuiBrilho(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][1][2]:
      return True
    return False
  
  @staticmethod
  def possuiImpacto(self, linha, coluna):
    impacto = self.status_posicao[f"{linha}_{coluna}_posicao"][1][3]
    if impacto:
      return impacto
    return False
  
  @staticmethod
  def possuiOK(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    for tupla in status:
      for elemento in tupla:
        if elemento != 0:
          return False
    return True

# Modelagem do Mundo de Wumpus para Solver 👾

In [ ]:
class Solver():
  def __init__(self, linhas, colunas, base_de_conhecimento):
    self.glucose = Glucose3()
    self.linhas = linhas
    self.colunas = colunas
    self.base_de_conhecimento = base_de_conhecimento
    self.mapeamento = {}
    self.mapeamento_reverso = {}
    self.index = 1
    self.mapear(self)
    self.atomo = ""
    self.atomo_cima = ""
    self.atomo_esquerda = ""
    self.atomo_direita = ""
    self.atomo_baixo = ""
    self.regrasDeMapa(self)

  @staticmethod
  def regrasDeMapa(self):
    for linha in range(0, self.linhas):
      for coluna in range(0, self.colunas):
        atomo1 = self.mapeamento[f"{linha}_{coluna}_wumpus"]
        atomo2 = self.mapeamento[f"{linha}_{coluna}_poco"]
        atomo3 = self.mapeamento[f"{linha}_{coluna}_ouro"]

        self.glucose.add_clause([-atomo1, -atomo2])
        self.glucose.add_clause([-atomo1, -atomo3])
        self.glucose.add_clause([-atomo2, -atomo3])

    for linha in [-1, 0, self.linhas - 1, self.linhas]:
      for coluna in range(-1, self.colunas + 1):
        try: atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]
        except: 
          self.mapearInstancia(self, linha, coluna, "parede")
          atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]

        try: atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]
        except: 
          self.mapearInstancia(self, linha, coluna, "impacto")
          atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]

        self.glucose.add_clause([-atomo1, -atomo2])

    for linha in range(-1, self.linhas):
      for coluna in [-1, 0, self.linhas - 1, self.linhas]:
        try: atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]
        except: 
          self.mapearInstancia(self, linha, coluna, "parede")
          atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]

        try: atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]
        except: 
          self.mapearInstancia(self, linha, coluna, "impacto")
          atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]

        self.glucose.add_clause([-atomo1, -atomo2])
    
    self.glucose.add_clause([self.mapeamento[f"-1_0_parede"]])
    self.glucose.add_clause([self.mapeamento[f"0_-1_parede"]])
    # self.mapearInstancia(self, 1, 1, "parede")
    # self.glucose.add_clause([self.mapeamento[f"- 1_1_parede"]])
    # self.glucose.add_clause([self.mapeamento[f"- 0_0_wumpus"]])
    # self.glucose.add_clause([self.mapeamento[f"- 0_0_poco"]])
    # self.glucose.add_clause([self.mapeamento[f"- 0_0_ouro"]])
  
  @staticmethod
  def mapearInstancia(self, linha, coluna, objeto):
    self.mapeamento[f"{linha}_{coluna}_{objeto}"] = self.index
    self.mapeamento[f"- {linha}_{coluna}_{objeto}"] = -self.index
    self.mapeamento_reverso[self.index] = f"{linha}_{coluna}_{objeto}"
    self.mapeamento_reverso[-self.index] = f"- {linha}_{coluna}_{objeto}"

    self.index += 1

  @staticmethod
  def mapear(self):
    for linha in range(0, self.linhas):
      for coluna in range(0, self.colunas):
        self.mapearInstancia(self, linha, coluna, "wumpus")
        self.mapearInstancia(self, linha, coluna, "poco")
        self.mapearInstancia(self, linha, coluna, "ouro")
        self.mapearInstancia(self, linha, coluna, "fedor")
        self.mapearInstancia(self, linha, coluna, "brisa")
        self.mapearInstancia(self, linha, coluna, "brilho")
        self.mapearInstancia(self, linha, coluna, "impacto")
    
    for linha in [0, self.linhas - 1]:
      for coluna in range(0, self.colunas):
        self.mapearInstancia(self, linha, coluna, "impacto")

    for linha in range(0, self.linhas):
      for coluna in [0, self.colunas - 1]:
        self.mapearInstancia(self, linha, coluna, "impacto")
    
    for linha in [-1, self.linhas]:
      for coluna in range(-1, self.colunas + 1):
        self.mapearInstancia(self, linha, coluna, "parede")

    for linha in range(-1, self.linhas + 1):
      for coluna in [-1, self.colunas]:
        self.mapearInstancia(self, linha, coluna, "parede")

    self.mapeamento[f"verdade"] = self.index
    self.mapeamento_reverso[self.index] = f"verdade"

    self.glucose.add_clause([self.index])

    self.mapeamento[f"falso"] = -self.index
    self.mapeamento_reverso[-self.index] = f"falso"

  def imprimirMapeamento(self):
    print("mapeamento[posição] -> index")
    print(self.mapeamento)
    print("mapeamento_reverso[index] -> posição")
    print(self.mapeamento_reverso)
  
  def perguntarRegra(self, regra):
    try: regra = self.mapeamento[regra]
    except: 
      self.mapeamento[f"{regra}"] = self.index
      self.mapeamento[f"- {regra}"] = -self.index
      self.mapeamento_reverso[self.index] = f"{regra}"
      self.mapeamento_reverso[-self.index] = f"- {regra}"

      self.index += 1
      regra = self.mapeamento[regra]

    if self.glucose.solve(assumptions=[regra]): return True
    return False
  
  @staticmethod
  def inserirConhecimento(self, conhecimento):
    self.base_de_conhecimento.inserirConhecimento(f"{self.mapeamento_reverso[conhecimento]}")

  @staticmethod
  def inserirRaciocinio(self, raciocinio):
    regra = ""
    for atomo in raciocinio:
      regra += f"{self.mapeamento_reverso[atomo]}, "
    regra = regra[0:-2]
    self.base_de_conhecimento.inserirRaciocinio(regra)
  
  def resolverSolve(self):
    self.glucose.solve()

  def retornarSolve(self):
    self.glucose.get_model()
  
  def imprimirSolve(self):
    print(self.glucose.get_model())
  
  def adicionarRegra(self, regra):
    self.mapeamento[f"{regra}"] = self.index
    self.mapeamento[f"- {regra}"] = -self.index
    self.mapeamento_reverso[self.index] = f"{regra}"
    self.mapeamento_reverso[-self.index] = f"- {regra}"
    self.index += 1

    self.atomo = self.mapeamento[f"{regra}"]
    self.eVerdade(self)

  # Equivalências Lógicas

  @staticmethod
  def eVerdade(self):
    self.glucose.add_clause([self.atomo])

    self.inserirConhecimento(self, self.atomo)

  @staticmethod
  def eFalso(self):
    self.glucose.add_clause([-self.atomo])

    self.inserirConhecimento(self, -self.atomo)
  
  @staticmethod
  def naoPossui(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima])
    self.glucose.add_clause([-self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita])

    self.inserirConhecimento(self, -self.atomo_cima)
    self.inserirConhecimento(self, -self.atomo_esquerda)
    self.inserirConhecimento(self, -self.atomo_baixo)
    self.inserirConhecimento(self, -self.atomo_direita)

  @staticmethod
  def possui1Verdadeiro(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_baixo, -self.atomo_direita])
  
  @staticmethod
  def possui2Verdadeiro(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

  @staticmethod
  def possui3Verdadeiro(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_direita])
  
  @staticmethod
  def impactosAdjacentesHorizontal(self, linha, coluna):
    # ((p&e)#(p&d))_impacto>((c&-b)#(-c&b))_parede
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, "parede")
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_impacto"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, "impacto")
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_impacto"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, "parede")
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_impacto"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, "impacto")
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_impacto"]

    self.glucose.add_clause([-self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])  

    self.inserirRaciocinio(self, [-self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
  
  @staticmethod
  def impactosAdjacentesVertical(self, linha, coluna):

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_impacto"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, "impacto")
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_impacto"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, "parede")
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_impacto"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, "impacto")
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_impacto"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, "parede")
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]

    self.glucose.add_clause([-self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita]) 

    self.inserirRaciocinio(self, [-self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])   

  def naoPossuiFedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eFalso(self)

    self.naoPossui(self, linha, coluna, "wumpus")
    
  def naoPossuiBrisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eFalso(self)

    self.naoPossui(self, linha, coluna, "poco")
    
  def possui1Fedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eVerdade(self)

    self.possui1Verdadeiro(self, linha, coluna, "wumpus")

  def possui1Brisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eVerdade(self)

    self.possui1Verdadeiro(self, linha, coluna, "poco")
     
  def possui2Fedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eVerdade(self)

    self.possui2Verdadeiro(self, linha, coluna, "wumpus")

  def possui2Brisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eVerdade(self)

    self.possui2Verdadeiro(self, linha, coluna, "poco")

  def possui3Fedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eVerdade(self)

    self.possui3Verdadeiro(self, linha, coluna, "wumpus")

  def possui3Brisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eVerdade(self)
    
    self.possui3Verdadeiro(self, linha, coluna, "poco")

  def naoPossuiBrilho(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brilho"]
    self.eFalso(self)

    self.atomo = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.eFalso(self)
  
  def possuiBrilho(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brilho"]
    self.eVerdade(self)

    self.atomo = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.eVerdade(self)
  
  def naoPossuiImpacto(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.eFalso(self)
    
    self.naoPossui(self, linha, coluna, "parede")

  def possui1Impacto(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.eVerdade(self)

    self.possui1Verdadeiro(self, linha, coluna, "parede")

    self.impactosAdjacentesVertical(self, linha, coluna)

    self.impactosAdjacentesHorizontal(self, linha, coluna)

  def possui2Impacto(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.eVerdade(self)

    self.possui2Verdadeiro(self, linha, coluna, "parede")
    
    self.impactosAdjacentesVertical(self, linha, coluna)

    self.impactosAdjacentesHorizontal(self, linha, coluna)

# Função do Agente

In [ ]:
class Agente():
  def __init__(self, solver, base_de_conhecimento):
    self.solver = solver
    self.base_de_conhecimento = base_de_conhecimento
    self.linha = 0
    self.coluna = 0
    self.max_linha = 0
    self.max_coluna = 0
    self.orientacao = 0
    self.posicoes = []
    self.desempenho = 0
    self.status_agente = "explorando"
    self.pilha_de_exploracao = []
    self.proxima_direcao = []

  def definirDesempenho(self, desempenho):
    self.desempenho = desempenho
  
  def definirPilhaDeExploracao(self, pilha_de_exploracao):
    self.pilha_de_exploracao = pilha_de_exploracao
  
  def definirLinhaColuna(self, linha, coluna):
    self.linha = linha
    self.coluna = coluna
  
  def definirOrientacao(self, orientacao):
    self.orientacao = orientacao
  
  def Agente(self, percepcao):
    self.inferir(self, percepcao)
    self.criarNovoConhecimento(self)
    acao = self.escolherAcao(self)
    return acao
  
  @staticmethod
  def inferir(self, percepcao):
    fedor = percepcao[1][0]
    brisa = percepcao[1][1]
    brilho = percepcao[1][2]
    impacto = percepcao[1][3]
    # grito = self.percepcao[1][4]

    if fedor == 1: self.solver.possui1Fedor(self.linha, self.coluna)
    elif fedor == 2: self.solver.possui2Fedor(self.linha, self.coluna)
    elif fedor == 3: self.solver.possui3Fedor(self.linha, self.coluna)
    else: self.solver.naoPossuiFedor(self.linha, self.coluna)

    if brisa == 1: self.solver.possui1Brisa(self.linha, self.coluna)
    elif brisa == 2: self.solver.possui2Brisa(self.linha, self.coluna)
    elif brisa == 3: self.solver.possui3Brisa(self.linha, self.coluna)
    else: self.solver.naoPossuiBrisa(self.linha, self.coluna)

    if impacto == 1: self.solver.possui1Impacto(self.linha, self.coluna)
    elif impacto == 2: self.solver.possui2Impacto(self.linha, self.coluna)
    else: self.solver.naoPossuiImpacto(self.linha, self.coluna)

    if brilho: self.solver.possuiBrilho(self.linha, self.coluna)
    else: self.solver.naoPossuiBrilho(self.linha, self.coluna)

    posicao_explorada = f"{self.linha}_{self.coluna}_explorado"
    self.solver.adicionarRegra(posicao_explorada)

    self.max_linha = max(self.max_linha, self.linha)
    self.max_coluna = max(self.max_coluna, self.coluna)
    self.posicoes.append((self.linha, self.coluna))
  
  @staticmethod
  def criarNovoConhecimento(self):
    conhecimento = self.base_de_conhecimento.retornarConhecimento()
    novo_conhecimento = []

    for linha in range(0, self.max_linha + 1):
      for coluna in range(0, self.max_coluna + 1):
        # if (linha, coluna) in self.posicoes:

        regra = f"{linha}_{coluna}_wumpus"
        if self.perguntarRegraComCerteza(self, regra): novo_conhecimento.append(f"{regra}")
        elif self.perguntarRegraComCerteza(self, f"- {regra}"): novo_conhecimento.append(f"- {regra}")
        
        regra = f"{linha}_{coluna}_poco"
        if self.perguntarRegraComCerteza(self, regra): novo_conhecimento.append(f"{regra}")
        elif self.perguntarRegraComCerteza(self, f"- {regra}"): novo_conhecimento.append(f"- {regra}")

        regra = f"{linha}_{coluna}_ouro"
        if self.perguntarRegraComCerteza(self, regra): novo_conhecimento.append(f"{regra}")
        elif self.perguntarRegraComCerteza(self, f"- {regra}"): novo_conhecimento.append(f"- {regra}")

        regra = f"{linha}_{coluna}_parede"
        if self.perguntarRegraComCerteza(self, regra): novo_conhecimento.append(f"{regra}")
        elif self.perguntarRegraComCerteza(self, f"- {regra}"): novo_conhecimento.append(f"- {regra}")
      
    for atomo in novo_conhecimento: 
      if not atomo in conhecimento:
        self.base_de_conhecimento.inserirConhecimento(atomo)
        # não adicionar ao solver
  
  @staticmethod
  def escolherAcao(self):
    explorar = self.explorar(self)

    if self.perguntarSairDaCaverna(self): return "saindo_da_caverna"
    elif self.perguntarOuro(self): return "pegar_ouro"
    elif explorar: return explorar
    else: return "retornar"
  
  @staticmethod
  def perguntarSairDaCaverna(self): 
    if self.desempenho > 0: return True
    elif self.desempenho < 0 and len(self.pilha_de_exploracao) == 1 and not self.proxima_direcao: return True
    else: return False

  @staticmethod
  def perguntarOuro(self):
    ouro = self.solver.perguntarRegra(f"{self.linha}_{self.coluna}_ouro")
    pego = f"{self.linha}_{self.coluna}_ouro_pego"
    pego = self.perguntarRegraComCerteza(self, pego)
    if ouro and not pego: return True
    else: return False
  
  @staticmethod
  def explorar(self):
    # Definir direções
    busca = {
        # direcao: (pode_avancar, foi_explorado)
        "frente": (False, True),
        "esquerda": (False, True),
        "atras": (False, True),
        "direita": (False, True)        
    }

    for direcao in ["frente", "esquerda", "atras", "direita"]:
      pode_avancar = False
      foi_explorado = True

      if self.peguntarAvancar(self): pode_avancar = True
      if not self.peguntarExplorado(self): foi_explorado = False
      
      busca[direcao] = (pode_avancar, foi_explorado)
      self.incrementarOrientacao(self)

    print(busca)

    # Decidir para onde vai 
    proxima_direcao = [] # Prioridade: (True, False)
    for direcao in ["frente", "esquerda", "direita"]:
      if busca[direcao] == (True, False):
        proxima_direcao.append(direcao) 
    
    self.proxima_direcao = proxima_direcao # perguntarSairDaCaverna

    if proxima_direcao: 
      proxima_direcao = choice(proxima_direcao)
      return proxima_direcao
    else: return False

  @staticmethod
  def perguntarRegraComCerteza(self, regra):
    # Verdadeira apenas se houver certeza
    if "-" in regra:
      regra = self.solver.perguntarRegra(f"{regra}") and not self.solver.perguntarRegra(f"{regra[2:]}")
      if regra: return True
      else: return False
    else:
      regra = self.solver.perguntarRegra(f"{regra}") and not self.solver.perguntarRegra(f"- {regra}")
      if regra: return True
      else: return False

  @staticmethod
  def peguntarAvancar(self):
    linha, coluna = self.posicaoAFrente(self)

    try:
      wumpus = self.solver.perguntarRegra(f"{linha}_{coluna}_wumpus")
      poco = self.solver.perguntarRegra(f"{linha}_{coluna}_poco")
      parede = self.solver.perguntarRegra(f"{linha}_{coluna}_parede")

      return not (wumpus or poco or parede)
    except: return False
  
  @staticmethod
  def peguntarExplorado(self):
    linha, coluna = self.posicaoAFrente(self)
    explorado = not (self.solver.perguntarRegra(f"{linha}_{coluna}_explorado") and self.solver.perguntarRegra(f"- {linha}_{coluna}_explorado"))
    return explorado
    try: 
      if self.solver.perguntarRegra(f"{linha}_{coluna}_explorado"):
        if self.solver.perguntarRegra(f"- {linha}_{coluna}_explorado"): return False
        else: return True
      else: return False
    except: return False

  @staticmethod
  def posicaoAFrente(self):
    linha = self.linha
    coluna = self.coluna
    if self.orientacao == 0: coluna += 1
    elif self.orientacao == 90: linha -= 1
    elif self.orientacao == 180: coluna -= 1
    elif self.orientacao == 270: linha += 1
    return linha, coluna

  @staticmethod
  def incrementarOrientacao(self):
    self.orientacao += 90
    if self.orientacao == 360: self.orientacao = 0
  
  @staticmethod
  def decrementarOrientacao(self):
    self.orientacao -= 90
    if self.orientacao == -90: self.orientacao = 270

# Mundo de Wumpus

In [452]:
class MundoDeWumpus():
  def __init__(self):
    self.mapa = Mapa(linhas = 4, colunas = 4, quantidade_wumpus = 1, quantidade_pocos = 2, quantidade_ouros = 2)
    self.mapa.gerarMapa()
    self.base_de_conhecimento = BaseDeConhecimento()
    self.solver = Solver(4, 4, self.base_de_conhecimento)
    self.agente = Agente(self.solver, self.base_de_conhecimento)
    self.linha = 0
    self.coluna = 0
    self.orientacao = 0 # ângulo
    self.desempenho = 0
    self.status_agente = "explorando"
    self.pilha_de_exploracao = []
  
  def imprimirMapa(self):
    self.mapa.imprimirMapa()
  
  def resolverMundoDeWumpus(self):
    while self.status_agente == "explorando":
    #if self.status_agente == "explorando":
      self.empilharPosicao(self)
      self.agente.definirDesempenho(self.desempenho)
      self.agente.definirPilhaDeExploracao(self.pilha_de_exploracao)
      self.agente.definirLinhaColuna(self.linha, self.coluna)
      self.agente.definirOrientacao(self.orientacao)

      percepcao = self.receberSensor(self)
      acao = self.agente.Agente(percepcao)
      self.base_de_conhecimento.inserirAcao(acao)

      print(f"posicao: {self.agente.linha}_{self.agente.coluna}") 
      try: print(f"percepcao: {percepcao}") 
      except: pass
      print(f"desenpenho: {self.desempenho} status_argente: {self.status_agente}")
      print(f"pilha_de_exploracao: {self.pilha_de_exploracao}")

      self.atuadores(self, acao)

      self.validarVida(self, percepcao)
  
  @staticmethod
  def empilharPosicao(self):
    posicao_atual = (self.linha, self.coluna)
    if self.pilha_de_exploracao:
      ultima_posicao = self.pilha_de_exploracao[-1]
      linha = ultima_posicao[0]
      coluna = ultima_posicao[1]
      ultima_posicao = (linha, coluna)
      if ultima_posicao == posicao_atual: pass
      else: self.pilha_de_exploracao.append(posicao_atual)
    else: self.pilha_de_exploracao.append(posicao_atual)
  
  @staticmethod
  def validarVida(self, percepcao):
    wumpus = percepcao[0][0]
    poco = percepcao[0][1]
    parede = percepcao[0][3]

    if wumpus or poco or parede: 
      self.status_agente == "morto"
      print("Agente morto")
      exit(1)
  
  @staticmethod
  def atuadores(self, acao):
    if acao == "saindo_da_caverna":
      if self.sairDaCaverna(self): 
        print("saiu da caverna")
        self.status_agente = "fim"
        self.base_de_conhecimento.inserirAcao("sair_da_caverna")
      else: self.caminharParaVoltar(self)      
    elif acao == "pegar_ouro": self.pegarOuro(self)
    elif acao == "frente": self.avancar(self)
    elif acao == "esquerda": 
      self.girarParaEsquerda(self)
      self.base_de_conhecimento.inserirAcao("avancar")
      self.avancar(self)
    elif acao == "direita":
      self.girarParaDireita(self)
      self.base_de_conhecimento.inserirAcao("avancar")
      self.avancar(self)
    else: 
      self.caminharParaVoltar(self) 
  
  def imprimirMapaDoConhecimento(self):
    mapa = []
    init() # colorama
    for linha in range(-1, 5):
      elementos_da_linha = []
      for coluna in range(-1, 5):
        elementos_da_celula = ""

        if linha == self.linha and coluna == self.coluna:
          elementos_da_celula += (Fore.WHITE + f"agente{self.orientacao}")

        if self.solver.perguntarRegra(f"{linha}_{coluna}_wumpus"):
          elementos_da_celula += (Fore.GREEN + "wumpus")
        if self.solver.perguntarRegra(f"{linha}_{coluna}_poco"):
          elementos_da_celula += (Fore.RED + "poco")
        if self.solver.perguntarRegra(f"{linha}_{coluna}_ouro"):
          elementos_da_celula += (Fore.YELLOW + "ouro")
        if self.solver.perguntarRegra(f"{linha}_{coluna}_parede"):
          elementos_da_celula += (Fore.MAGENTA + "parede")
        if self.solver.perguntarRegra(f"{linha}_{coluna}_fedor"):
          elementos_da_celula += (Fore.GREEN + "fedor")
        if self.solver.perguntarRegra(f"{linha}_{coluna}_brisa"):
          elementos_da_celula += (Fore.RED + "brisa")
        if self.solver.perguntarRegra(f"{linha}_{coluna}_brilho"):
          elementos_da_celula += (Fore.YELLOW + "brilho")
        if self.solver.perguntarRegra(f"{linha}_{coluna}_impacto"):
          elementos_da_celula += (Fore.MAGENTA + "impacto")
        
        if elementos_da_celula == "": elementos_da_celula += (Fore.WHITE + "ok") 
        
        elementos_da_linha.append(elementos_da_celula)
      mapa.append(elementos_da_linha)
    deinit() # colorama
    print(tabulate(mapa, tablefmt="plain", stralign="center"))
    print(Fore.WHITE) # gambiarra
  
  def imprimirBaseDeConhecimento(self):
    self.base_de_conhecimento.imprimirTabela()
  
  @staticmethod
  def receberSensor(self):
    return self.mapa.retornarStatusDaPosica(self.linha, self.coluna)  

  @staticmethod
  def sairDaCaverna(self):
    if self.linha == 0 and self.coluna == 0:
      self.desempenho -= 1
      return True
    else: return False
  
  @staticmethod
  def caminharParaVoltar(self):
    try:
      self.pilha_de_exploracao.pop() # Posição atual
      ultima_posicao = self.pilha_de_exploracao.pop()
      linha = ultima_posicao[0]
      coluna = ultima_posicao[1]

      if linha < self.linha: self.girarParaOrientacao(self, 90)
      elif linha > self.linha: self.girarParaOrientacao(self, 270)
      elif coluna < self.coluna: self.girarParaOrientacao(self, 180)
      elif coluna > self.coluna: self.girarParaOrientacao(self, 0)

      self.avancar(self)
    except: 
      print("saiu da caverna")
      self.status_agente = "fim"
      self.pilha_de_exploracao = []

  @staticmethod
  def pegarOuro(self):
    self.solver.adicionarRegra(f"{self.linha}_{self.coluna}_ouro_pego")
    self.desempenho += 1000
  
  @staticmethod
  def avancar(self):
    if self.orientacao == 0: self.coluna += 1
    elif self.orientacao == 90: self.linha -= 1
    elif self.orientacao == 180: self.coluna -= 1
    elif self.orientacao == 270: self.linha += 1
    self.desempenho -= 1

  @staticmethod
  def girarParaOrientacao(self, orientacao):
    diferenca = orientacao - self.orientacao
    print(f"diferenca = orientacao - self.orientacao: {diferenca} = {orientacao} - {self.orientacao}")

    if diferenca == 0: pass
    elif diferenca == 90: 
      self.girarParaEsquerda(self)
    elif diferenca == -90: 
      self.girarParaDireita(self)
    elif diferenca == 180:
      self.girarParaEsquerda(self)
      self.girarParaEsquerda(self)
    elif diferenca == -180:
      self.girarParaEsquerda(self)
      self.girarParaEsquerda(self)
    elif diferenca == 270:
      self.girarParaDireita(self)
    elif diferenca == -270:
      self.girarParaEsquerda(self)

  @staticmethod
  def girarParaEsquerda(self):
    self.incrementarOrientacao(self)
    self.desempenho -= 1
  
  @staticmethod
  def girarParaDireita(self):
    self.decrementarOrientacao(self)
    self.desempenho -= 1
  
  @staticmethod
  def incrementarOrientacao(self):
    self.orientacao += 90
    if self.orientacao == 360: self.orientacao = 0
  
  @staticmethod
  def decrementarOrientacao(self):
    self.orientacao -= 90
    if self.orientacao == -90: self.orientacao = 270


# Main

In [480]:
mundo_de_wumpus = MundoDeWumpus()
mundo_de_wumpus.imprimirMapa()


parede           parede               parede       parede            parede           parede
parede          2impacto          1impacto1brisa  1impacto       2impacto1brisa       parede
parede  1impactoouro1brisabrilho       poco        2brisa         1impactopoco        parede
parede          1impacto              1brisa         ok     1impactoouro1brisabrilho  parede
parede          2impacto             1impacto     1impacto          2impacto          parede
parede           parede               parede       parede            parede           parede



In [ ]:
mundo_de_wumpus.resolverMundoDeWumpus()
mundo_de_wumpus.imprimirMapaDoConhecimento()

In [ ]:
mundo_de_wumpus.imprimirBaseDeConhecimento()